# Springboard Capstone Project 2
## Training models on the downsampled image set
___

There can be high variance in performance among instances of the same model trained on the same image set, due to the random initialization of weights. For this reason, multiple models were trained on the downsampled image set, and the best-performing model after 10 epochs was trained until convergence.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import load_model
import importlib as imp
import gc
from datetime import datetime

# custom module for capstone 2
import cap2tools as c2t
imp.reload(c2t)

Using TensorFlow backend.


<module 'cap2tools' from 'C:\\Users\\Nils\\Documents\\GitHub\\Springboard-Capstone-2-local-yelp\\cap2tools.py'>

In [2]:
# configure GPU memory usage by tensorflow
config = K.tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.90
K.tensorflow_backend.set_session(K.tf.Session(config=config))

In [3]:
# define paths to image directories
photos_path = 'C:/Users/Nils/Documents/GitHub/Springboard-Capstone-2-local-yelp/downsampled/'
train_path = photos_path + 'train'
valid_path = photos_path + 'val'

# create data generators
train_batches, valid_batches = c2t.build_datagens(train_path, valid_path, augment=True)

Found 5480 images belonging to 5 classes.
Found 525 images belonging to 5 classes.


In [4]:
replicates = 10
n_epochs = 10
histories = dict()

for i in range(1, replicates+1):
    
    # build model
    model = c2t.build_VGG16(widths=(1000, 1250), 
                            new_weights=False, 
                            trainable=True, 
                            dropout1=0, 
                            dropout2=0.2)

    model.compile(optimizer=Adam(lr=0.0001, decay=0.1), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    # define callbacks
    filepath = 'models/downsample_model_{}.h5'.format(i)
    saver = ModelCheckpoint(filepath,
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True)

    stopper = EarlyStopping(monitor='val_loss', 
                            patience=2, 
                            verbose=1)

    datetime_now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print('{} - Started training {}'.format(datetime_now, filepath))
    
    # train model
    history = model.fit_generator(train_batches, 
                                  validation_data=valid_batches,
                                  epochs=n_epochs, 
                                  verbose=0, 
                                  callbacks=[saver, stopper])
    
    histories[i] = history.history
    
    # clear memory
    K.clear_session()
    del model
    gc.collect()

2018-09-21 00:49:43 - Started training models/downsample_model_1.h5

Epoch 00001: val_loss improved from inf to 0.46532, saving model to models/downsample_model_1.h5

Epoch 00002: val_loss improved from 0.46532 to 0.39715, saving model to models/downsample_model_1.h5

Epoch 00003: val_loss improved from 0.39715 to 0.37215, saving model to models/downsample_model_1.h5

Epoch 00004: val_loss improved from 0.37215 to 0.34887, saving model to models/downsample_model_1.h5

Epoch 00005: val_loss improved from 0.34887 to 0.32897, saving model to models/downsample_model_1.h5

Epoch 00006: val_loss improved from 0.32897 to 0.32777, saving model to models/downsample_model_1.h5

Epoch 00007: val_loss improved from 0.32777 to 0.31145, saving model to models/downsample_model_1.h5

Epoch 00008: val_loss improved from 0.31145 to 0.31048, saving model to models/downsample_model_1.h5

Epoch 00009: val_loss improved from 0.31048 to 0.30298, saving model to models/downsample_model_1.h5

Epoch 00010: val_


Epoch 00001: val_loss improved from inf to 0.50460, saving model to models/downsample_model_10.h5

Epoch 00002: val_loss improved from 0.50460 to 0.37650, saving model to models/downsample_model_10.h5

Epoch 00003: val_loss improved from 0.37650 to 0.36725, saving model to models/downsample_model_10.h5

Epoch 00004: val_loss improved from 0.36725 to 0.33201, saving model to models/downsample_model_10.h5

Epoch 00005: val_loss improved from 0.33201 to 0.32967, saving model to models/downsample_model_10.h5

Epoch 00006: val_loss improved from 0.32967 to 0.32074, saving model to models/downsample_model_10.h5

Epoch 00007: val_loss did not improve from 0.32074

Epoch 00008: val_loss did not improve from 0.32074
Epoch 00008: early stopping


In [5]:
# evaluate model on downsampled val set
model_paths = dict()
for i in range(1, replicates+1):
    value = 'models/downsample_model_{}.h5'.format(str(i))
    
    model_paths[i] = value
    
model_metrics = c2t.eval_models(model_paths, valid_path)

Building image generator...
Found 525 images belonging to 5 classes.
Loading models/downsample_model_1.h5
Evaluating models/downsample_model_1.h5
Loading models/downsample_model_2.h5
Evaluating models/downsample_model_2.h5
Loading models/downsample_model_3.h5
Evaluating models/downsample_model_3.h5
Loading models/downsample_model_4.h5
Evaluating models/downsample_model_4.h5
Loading models/downsample_model_5.h5
Evaluating models/downsample_model_5.h5
Loading models/downsample_model_6.h5
Evaluating models/downsample_model_6.h5
Loading models/downsample_model_7.h5
Evaluating models/downsample_model_7.h5
Loading models/downsample_model_8.h5
Evaluating models/downsample_model_8.h5
Loading models/downsample_model_9.h5
Evaluating models/downsample_model_9.h5
Loading models/downsample_model_10.h5
Evaluating models/downsample_model_10.h5
Evaluation complete.



In [7]:
model_metrics

{1: {'loss': 0.2988273965070645,
  'acc': 0.8952380952380953,
  'cm': [[89, 4, 5, 1, 6],
   [5, 98, 0, 1, 1],
   [5, 2, 89, 2, 7],
   [0, 0, 0, 104, 1],
   [0, 0, 14, 1, 90]],
  'pcr': array([0.84761905, 0.93333333, 0.84761905, 0.99047619, 0.85714286]),
  'mpcr': 0.8952380952380953},
 2: {'loss': 0.376645097551823,
  'acc': 0.8361904764175415,
  'cm': [[79, 8, 12, 0, 6],
   [9, 91, 1, 2, 2],
   [8, 1, 79, 3, 14],
   [1, 0, 0, 102, 2],
   [1, 0, 15, 1, 88]],
  'pcr': array([0.75238095, 0.86666667, 0.75238095, 0.97142857, 0.83809524]),
  'mpcr': 0.8361904761904763},
 3: {'loss': 0.2991715855140328,
  'acc': 0.885714285941351,
  'cm': [[87, 6, 8, 1, 3],
   [3, 98, 0, 2, 2],
   [6, 3, 81, 1, 14],
   [1, 0, 0, 102, 2],
   [0, 0, 8, 0, 97]],
  'pcr': array([0.82857143, 0.93333333, 0.77142857, 0.97142857, 0.92380952]),
  'mpcr': 0.8857142857142858},
 4: {'loss': 0.37192804755035985,
  'acc': 0.845714285941351,
  'cm': [[77, 3, 19, 3, 3],
   [2, 101, 0, 1, 1],
   [6, 1, 71, 1, 26],
   [1, 0, 0

In [6]:
# train model 1 until convergence
model = load_model('models/downsample_model_1.h5')

# define callbacks
filepath = 'models/downsample_model_1.h5'
saver = ModelCheckpoint(filepath,
                        monitor='val_loss',
                        verbose=1,
                        save_best_only=True)

stopper = EarlyStopping(monitor='val_loss', 
                        patience=3, 
                        verbose=1)

history = model.fit_generator(train_batches, 
                              validation_data=valid_batches,
                              epochs=30, 
                              verbose=1, 
                              callbacks=[saver, stopper])

# clear memory
K.clear_session()
del model
gc.collect()

Epoch 1/30
685/685 [==============================] - 156s 228ms/step - loss: 0.3869 - acc: 0.8491 - val_loss: 0.2918 - val_acc: 0.9010

Epoch 00001: val_loss improved from inf to 0.29180, saving model to models/downsample_model_1.h5
Epoch 2/30
685/685 [==============================] - 157s 229ms/step - loss: 0.3918 - acc: 0.8469 - val_loss: 0.2899 - val_acc: 0.9067

Epoch 00002: val_loss improved from 0.29180 to 0.28985, saving model to models/downsample_model_1.h5
Epoch 3/30
685/685 [==============================] - 157s 229ms/step - loss: 0.3700 - acc: 0.8584 - val_loss: 0.2871 - val_acc: 0.9067

Epoch 00003: val_loss improved from 0.28985 to 0.28706, saving model to models/downsample_model_1.h5
Epoch 4/30
685/685 [==============================] - 155s 226ms/step - loss: 0.3675 - acc: 0.8599 - val_loss: 0.2885 - val_acc: 0.9048

Epoch 00004: val_loss did not improve from 0.28706
Epoch 5/30
685/685 [==============================] - 155s 226ms/step - loss: 0.3627 - acc: 0.8578 - v

16790

In [7]:
# evaluate model on larger validation set
model_paths = {'model': 'models/downsample_model_1.h5'}
valid_path_large = 'H:/springboard/other_data/yelp/Photos/final_photos/val'
model_metrics_large = c2t.eval_models(model_paths, valid_path_large)['model']

Building image generator...
Found 20000 images belonging to 5 classes.
Loading models/downsample_model_1.h5
Evaluating models/downsample_model_1.h5
Evaluation complete.



In [8]:
c2t.print_eval(model_metrics_large)

accuracy:  88.94%
loss:  0.3005
pcr:  [0.8722 0.9197 0.8053 1.     0.8646]
mean pcr:  89.23%
confusion matrix: 


[[655, 17, 44, 7, 28],
 [697, 12179, 190, 70, 106],
 [92, 87, 3433, 44, 607],
 [0, 0, 0, 105, 0],
 [11, 2, 187, 22, 1417]]